In [3]:
import os


In [1]:
%pwd

'f:\\Projects\\e2e\\nlp_text_summarization\\research'

In [4]:
os.chdir("../")

In [5]:
%pwd

'f:\\Projects\\e2e\\nlp_text_summarization'

In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [7]:
from TextSummarizer.constants import *
from TextSummarizer.utils.common import read_yaml, create_directories

In [11]:
class ConfigurationManager:
    def __init__(self, 
                 config_filepath=CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_URL = config.source_URL,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir
        )

        return data_ingestion_config

In [8]:
import os
import urllib.request as request
import zipfile
from TextSummarizer.logging import logger
from TextSummarizer.utils.common import get_size

In [16]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, header = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{header}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")


    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """

        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [17]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e    

[2024-08-22 20:22:54,599: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-08-22 20:22:54,602: INFO: common: yaml file: params.yaml loaded successfully]
[2024-08-22 20:22:54,603: INFO: common: created directory at: artifacts]
[2024-08-22 20:22:54,605: INFO: common: created directory at: artifacts/data_ingestion]
[2024-08-22 20:22:56,624: INFO: 2067762338: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 7903314
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "151bf017b824e4b35c0cc97af5f859ae1e7101be85037ecd5b28b1aff70f13e8"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: E175:147AE6:55B74:58F87:66C780B5
Accept-Ranges: bytes
Date: Thu, 22 Aug 2024 18:22:57 GMT
Via: 1.1 varnish
X-Served-By: cache-mxp6964-MXP
X-Cache: HIT
